In [ ]:
import mne
import numpy as np
import csv
import os
from scipy.stats import skew, kurtosis
import antropy as ant
import pyeeg

def mean_variance(chunk):
    return np.mean(np.var(chunk, axis=0))

def mean_kurtosis(chunk):
    return np.mean(kurtosis(chunk, axis=0))

def mean_skewness(chunk):
    return np.mean(skew(chunk, axis=0))

def func_mean_hurst(chunk):
    return np.mean([pyeeg.hurst(chunk[i, :]) for i in range(chunk.shape[0])])

def mean_hjorth_mob_comp(chunk):
    mob_vals, comp_vals = zip(*[
        ant.hjorth_params(chunk[i, :]) if np.var(chunk[i, :]) > 0 else (0, 0)
        for i in range(chunk.shape[0])
    ])
    return np.mean(mob_vals), np.mean(comp_vals)

def all_psd(chunk, fs=256):
    N = chunk.shape[1]
    fft_vals = np.abs(np.fft.rfft(chunk, axis=1))
    psd_vals = (N / fs) * fft_vals ** 2
    fft_freq = np.fft.rfftfreq(N, 1.0 / fs)
    eeg_bands = {'Delta': (0, 4), 'Theta': (4, 8), 'Alpha': (8, 12), 'Beta': (12, 30), 'Gamma': (30, 45)}
    return [
        np.mean(psd_vals[:, np.where((fft_freq >= eeg_bands[band][0]) & (fft_freq <= eeg_bands[band][1]))[0]]) * 1e6
        for band in eeg_bands
    ]

def process_edf_in_chunks(file_path, chunk_size=5000):
    data = mne.io.read_raw_edf(file_path, preload=True)
    eeg_data = data.get_data()
    num_samples = eeg_data.shape[1]
    
    results = []
    for start in range(0, num_samples, chunk_size):
        end = min(start + chunk_size, num_samples)
        chunk = eeg_data[:, start:end]
        
        power_vals = all_psd(chunk)
        mean_var = mean_variance(chunk)
        mean_skew = mean_skewness(chunk)
        mean_h_mob, mean_h_comp = mean_hjorth_mob_comp(chunk)
        mean_hurst_val = func_mean_hurst(chunk)
        mean_k = mean_kurtosis(chunk)
        
        results.append(power_vals + [mean_skew, mean_var, mean_k, mean_h_mob, mean_h_comp, mean_hurst_val])
    
    return np.mean(results, axis=0)  # Aggregate over chunks

edf_folder = 'D:/Brain_siez_det/eegfiles/'
output_file = 'D:/Brain_siez_det/csvOutput/alloutput.csv'
edf_files = [f for f in os.listdir(edf_folder) if f.endswith('.edf')]

with open(output_file, 'w', newline='') as file:
    writer = csv.writer(file)
    header = ["File Name", "Delta", "Theta", "Alpha", "Beta", "Gamma", "Skewness", "Variance", "Kurtosis", "Hjorth_Mobility", "Hjorth_Complexity", "Hurst"]
    writer.writerow(header)
    
    for edf_file in edf_files:
        file_path = os.path.join(edf_folder, edf_file)
        extracted_features = process_edf_in_chunks(file_path)
        writer.writerow([edf_file] + extracted_features.tolist())

print("Chunk-based feature extraction completed and saved to", output_file)


Extracting EDF parameters from D:\Brain_siez_det\eegfiles\eeg1.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 1790207  =      0.000 ...  6992.996 secs...


d:\Brain_siez_det\pyeeg.py:85: RuntimeWarning: invalid value encountered in divide
  R_S = R_T / S_T
C:\Users\Yash Srivastava\AppData\Local\Temp\ipykernel_8572\2514187090.py:16: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  return np.mean(skew(chunk, axis=0))
C:\Users\Yash Srivastava\AppData\Local\Temp\ipykernel_8572\2514187090.py:13: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  return np.mean(kurtosis(chunk, axis=0))
d:\Brain_siez_det\pyeeg.py:85: RuntimeWarning: divide by zero encountered in divide
  R_S = R_T / S_T


Extracting EDF parameters from D:\Brain_siez_det\eegfiles\eeg10.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 1389311  =      0.000 ...  5426.996 secs...


C:\Users\Yash Srivastava\AppData\Local\Temp\ipykernel_8572\2514187090.py:40: RuntimeWarning: Channels contain different highpass filters. Highest filter setting will be stored.
  data = mne.io.read_raw_edf(file_path, preload=True)
C:\Users\Yash Srivastava\AppData\Local\Temp\ipykernel_8572\2514187090.py:40: RuntimeWarning: Channels contain different lowpass filters. Lowest filter setting will be stored.
  data = mne.io.read_raw_edf(file_path, preload=True)
